<a href="https://colab.research.google.com/github/ijnuegnaj/RAG_PRACTICE/blob/practice6/5_RAG_%E1%84%80%E1%85%AE%E1%84%92%E1%85%A7%E1%86%AB%E1%84%92%E1%85%A1%E1%84%80%E1%85%B5_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## RAG 구현    

검색 -> 증강 -> 생성(LLM을 빌려서 수행)     
- LLM이 언어를 생성할 때, 모르는 정보(질문과 관련된 정보)를 함께 주어 질문에 맞는 답변을 생성하도록 유도

#### 자연어 처리 스텝     
- 정제(정규표현식) -> 토큰화(언어를 한번에 이해 가능한 단위로 자름) -> 불용어(안쓰는 말) 제거 -> 표제어/어간 추출(동사원형 찾기 ex)먹었다 먹었니 먹을거야 먹지마 => '먹' / ex) play playing plays ...played => play)

#### RAG 스텝
문서 가져오기(문서 전처리) -> (자연어처리) -> 임베딩(글자->숫자) -> 벡터화('유사도 검색') -> 유사도 검색으로 '질문'과 비슷하게 생긴 부분을 찾아서 전달 => 전달받은 내용으로 LLM이 답변을 생성


In [ ]:
#langchain과 관련된 베이스 라이브러리들.
!pip install langchain openai langchain-community

#tiktoken은 언어의 '토큰화'를 지원해줌
!pip install tiktoken

#bm25는 벡터의 유사도 검색을 잘 해내는 라이브러리임
!pip install rank_bm25

#트랜스포머를 기반으로 문장이나 문서를 임베딩할 수 있도록 돕는 라이브러리
!pip install sentence-transformers

#벡터 데이터를 저장함(벡터 DB)
!pip install chromadb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 62.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 56.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 61.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7

## 주요 라이브러리 임포트

In [ ]:
import tiktoken, openai

#'임베딩'을 담당하는 라이브러리.
#허깅페이스의 특정 모델을 기반으로 임베딩을 도와주는 것이 HuggingFace~~~Embeddings 임
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.embeddings import HuggingFaceBgeEmbeddings

#문서를 가져온 후, 문서를 일정한 길이로 잘라줌(Text->Splitter)
from langchain.text_splitter import RecursiveCharacterTextSplitter

#VectorDB. 데이터를 임베딩한 후 숫자로 변한 데이터를 저장하고,
#DB내에서 숫자 데이터의 검색을 더 빠르게 할 수 있도록 지원함
from langchain.vectorstores import Chroma
#from langchain_community.vectorstores import FAISS > cpu용 FAISS, gpu용 FAISS

#pdf 파일이나 문서 파일을 입력했을 때, 내부의 데이터를 가져옴
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader

#웹사이트 텍스트를 가져오는 역할
from langchain.document_loaders import WebBaseLoader

In [ ]:
#토크나이저는 -> 토큰화를 시키는 것
#'cl100k_base' -> 토크나이저의 종류, gpt가 사용하는 토크나이저와 유사한 특징을 가짐
tokenizer = tiktoken.get_encoding('cl100k_base')

#인풋된 text가 정해진 토크나이저로 잘려 나가면, 그 길이를 알 수 있게 됨
def titoken_len(text):
  tokens = tokenizer.encode(text)
  return len(tokens)

#모델 다운로드
model_hug = HuggingFaceBgeEmbeddings(model_name='BAAI/bge-m3')

/tmp/ipython-input-2026074431.py:11: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  model_hug = HuggingFaceBgeEmbeddings(model_name='BAAI/bge-m3')
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [ ]:
#자료 모으기!!
loaders=[
    WebBaseLoader("https://github.com/huggingface"),
    WebBaseLoader("https://wikidocs.net/229987"),
    WebBaseLoader("https://en.wikipedia.org/wiki/Hugging_Face"),
]

In [ ]:
#데이터를 토큰화 하여 load 하기 위한 내용
docs = []

for load in loaders:
  #~~~Loader로 가져온 데이터에 대해 데이터를 불러오고 잘라줌 : load_and_split()
  docs.extend(load.load_and_split())

print(f'총 문서의 길이 : {len(docs)}')

총 문서의 길이 : 26


In [ ]:
docs[0]

Document(metadata={'source': 'https://github.com/huggingface', 'title': 'Hugging Face Â· GitHub', 'description': 'The AI community building the future. Hugging Face has 339 repositories available. Follow their code on GitHub.', 'language': 'en'}, page_content='Hugging Face Â· GitHub\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSkip to content\n\n\n\n\n\n\n\n\n\n\n\n\n\nNavigation Menu\n\nToggle navigation\n\n\n\n\n \n\n\n\n\n\n\n\n\n\n\n\n\n\n            Sign in\n          \n\n\n \n\n\nAppearance settings\n\n\n\n\n\n\nhuggingface\n\n\n\n\n\n\n\n        Product\n        \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n            GitHub Copilot\n          \n        Write better code with AI\n      \n\n\n\n\n\n\n\n\n            GitHub Spark\n              \n                New\n              \n\n        Build and deploy intelligent apps\n      \n\n\n\n\n\n\n\n\n            GitHub Models\n              \n                New\n              \n\n        Manage and 

In [ ]:
docs[15].page_content

'Textual: 모던 터미널 사용자 인터페이스 개발 도구\n                        \n\n\n\n\n\n                        \n                        The Fuck: 터미널에서 발생한 오타 수정 도구\n                        \n\n\n\n\n\n                        \n                        Typer: 파이썬 타입 힌트 기반 CLI 생성 도구\n                        \n\n\n\n\n\n                        \n                        alive-progress: 역동적인 진행 표시줄 도구\n                        \n\n\n\n\n\n                        \n                        argparse: 명령행 옵션, 인자와 부속 명령을 위한 파서\n                        \n\n\n\n\n\n                        \n                        asciimatics: 터미널 텍스트 기반 UI\n                        \n\n\n\n\n\n                        \n                        blessed: 터미널 애플리케이션 개발 도구\n                        \n\n\n\n\n\n                        \n                        click: 커맨드라인 인터페이스(CLI) 생성 라이브러리\n                        \n\n\n\n\n\n                        \n                        colorama: 터미널 텍스트의 색상과 스타일 라이브러리\n                        \n

In [ ]:
#토큰화(청크화)
#chunk_size = 문서(Document)를 얼마 단위로 자를까?
#chunk_overlap
#나는 코딩하는 것을 좋아합니다.
#나는 코딩하는 것을
#         하는 것을 좋아합니다.
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 250,
                                               chunk_overlap = 20,
                                               length_function = titoken_len)

texts = text_splitter.split_documents(docs)
print(f'자른 내용 : {texts[0]}, 총 길이 {len(texts)}')

자른 내용 : page_content='Hugging Face Â· GitHub













































Skip to content













Navigation Menu

Toggle navigation




 













            Sign in
          


 


Appearance settings






huggingface







        Product
        














            GitHub Copilot
          
        Write better code with AI
      








            GitHub Spark
              
                New
              

        Build and deploy intelligent apps
      








            GitHub Models
              
                New
              

        Manage and compare prompts
      








            GitHub Advanced Security
          
        Find and fix vulnerabilities
      








            Actions
          
        Automate any workflow
      














            Codespaces
          
        Instant dev environments
      








            Issues
          
        Plan and track work
      








            Code Revie

# 리트리버(검색)         

- 잘라놓은 텍스트 중에 내가 한 질문과 가장 유사한 내용을 담은 것은 무엇일까?          



In [ ]:
from langchain.retrievers import BM25Retriever

#BM25 TF-IDF(문서 빈도-역빈도)계열의 준수한 성능을 보이는 알고리즘
bm25_r = BM25Retriever.from_documents(texts)

#질문과 가장 닮은 내용을 담고 있는 top 3개를 뽑아라~
bm25_r.k = 3

In [ ]:
result = bm25_r.invoke('What is the huggingface?')
result[1].page_content

'Services and technologies[edit]\nTransformers Library[edit]\nThe Transformers library is a Python package that contains open-source implementations of transformer models for text, image, and audio tasks. It is mainly compatible with the PyTorch library, but previous versions were also compatible with TensorFlow and JAX deep learning libraries. It includes implementations of notable models like BERT and GPT-2.[20] The library was originally called "pytorch-pretrained-bert"[21] which was then renamed to "pytorch-transformers" and finally "transformers."\nA JavaScript version (Transformers.js[22]) has also been developed, allowing models to run directly in the browser through the ONNX runtime.\n\nHugging Face Hub[edit]\nThe Hugging Face Hub is a platform (centralized web service) for hosting:[23]'

## 리트리버가 모아온 정보를 GPT에 함께 전달!

In [ ]:
import os
from google.colab import userdata

os.environ['OPENAI_API_KEY'] = userdata.get('ssu')

#리트리버 관련 모듈
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.chat_models import ChatOpenAI

In [ ]:
#채팅할 대상인 llm 모델 객체 만듦
llm = ChatOpenAI()

#우리가 잘라 둔 text를 VectorDB에 넣어줌
#왜? 벡터(숫자)로 만들고 빠르게 '계산' 할 수 있게 하기 위해서 => '유사도' 검색
#Chroma dB에 texts를 model_hug 방법으로 임베딩해서, 넣어놓음
chroma_db = Chroma.from_documents(texts, model_hug)

/tmp/ipython-input-1522904487.py:2: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI()
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `XLMRobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [ ]:
#리트리버를 정의
multi_r = MultiQueryRetriever.from_llm(retriever=chroma_db.as_retriever(),
                                       llm=llm)

response = multi_r.invoke('What is the huggingface?')
print(response)

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `XLMRobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `XLMRobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `XLMRobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


[Document(metadata={'language': 'en', 'source': 'https://en.wikipedia.org/wiki/Hugging_Face', 'title': 'Hugging Face - Wikipedia'}, page_content='Category\n\n\n\n\n\nRetrieved from "https://en.wikipedia.org/w/index.php?title=Hugging_Face&oldid=1304314872"\nCategories: Machine learningOpen-source artificial intelligencePrivately held companies based in New York CityAmerican companies established in 20162016 establishments in New York CityHidden categories: Articles with short descriptionShort description is different from WikidataArticles lacking reliable references from February 2024All articles lacking reliable referencesArticles with a promotional tone from August 2025All articles with a promotional tone\n\n\n\n\n\n\n This page was last edited on 5 August 2025, at 08:19\xa0(UTC).\nText is available under the Creative Commons Attribution-ShareAlike 4.0 License;\nadditional terms may apply. By using this site, you agree to the Terms of Use and Privacy Policy. Wikipedia¬Æ is a registere

In [ ]:
response[0].page_content

'Category\n\n\n\n\n\nRetrieved from "https://en.wikipedia.org/w/index.php?title=Hugging_Face&oldid=1304314872"\nCategories: Machine learningOpen-source artificial intelligencePrivately held companies based in New York CityAmerican companies established in 20162016 establishments in New York CityHidden categories: Articles with short descriptionShort description is different from WikidataArticles lacking reliable references from February 2024All articles lacking reliable referencesArticles with a promotional tone from August 2025All articles with a promotional tone\n\n\n\n\n\n\n This page was last edited on 5 August 2025, at 08:19\xa0(UTC).\nText is available under the Creative Commons Attribution-ShareAlike 4.0 License;\nadditional terms may apply. By using this site, you agree to the Terms of Use and Privacy Policy. Wikipedia¬Æ is a registered trademark of the Wikimedia Foundation, Inc., a non-profit organization.\n\n\nPrivacy policy\nAbout Wikipedia\nDisclaimers\nContact Wikipedia\nC

In [ ]:
from langchain.chains import RetrievalQA

qa = RetrievalQA.from_chain_type(llm=llm, #이야기를 나눌 llm
                                 #map-reduce(데이터 개수가 많을 때), refine(순차적으로 인지할게)
                                  chain_type='stuff', #내가 넣어줄 도큐먼트를 어떻게 인지할거냐. stuff(한꺼번에 볼게)
                                  retriever=multi_r)

response = qa('What is the huggingface?')
print(response['result'])

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `XLMRobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `XLMRobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `XLMRobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Hugging Face is a company that provides tools and platforms to support the development and deployment of machine learning models. They contribute significantly to AI research and practical application development by making it easier for anyone to work with machine learning models. Their philosophy includes encouraging community collaboration and sharing knowledge and resources through open-source technologies.


In [ ]:
'''Hugging Face is a company that provides tools and platforms to support the development and deployment of machine learning models. They contribute significantly to AI research and practical application development by making it easier for anyone to work with machine learning models. Their philosophy includes encouraging community collaboration and sharing knowledge and resources through open-source technologies.

## PDF 기반의 RAG 실습        

1. 자료 가져옴(WebBase, Pdf, text, csv) ~~Loader/llamaIndex   
2. 토큰화(tiktoken), load_and_split => 데이터를 청크화 함   
3. 2의 내용을 VectorDB화     
4. 리트리버 -> 잘라둔 텍스트 덩어리를 '리트리버 알고리즘(bm25)' 에 던져서, '질문과 가장 유사한 텍스트를 고름'  
5. GPT에 질문(리트리버와 VectorDB를 함께 전달)    

In [ ]:
#langchain과 관련된 베이스 라이브러리들.
!pip install langchain openai langchain-community

#tiktoken은 언어의 '토큰화'를 지원해줌
!pip install tiktoken

#bm25는 벡터의 유사도 검색을 잘 해내는 라이브러리임
!pip install rank_bm25

#트랜스포머를 기반으로 문장이나 문서를 임베딩할 수 있도록 돕는 라이브러리
!pip install sentence-transformers

#벡터 데이터를 저장함(벡터 DB)
!pip install chromadb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 106.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 87.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.

In [ ]:
!pip install -q PyPDF2 faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 43.1 MB/s eta 0:00:00


In [ ]:
#VectorDB : faiss / faiss는 cpu, gpu 최적화 버전이 나누어져 있음
from langchain import FAISS
from PyPDF2 import PdfReader

#openai에게 질의응답
from langchain.chains.question_answering import load_qa_chain

#텍스트 스플리터
from langchain.text_splitter import CharacterTextSplitter

In [ ]:
#텍스트 전처리 함수
def preprocess_text(text):

  #텍스트 청크화
  text_splitter = CharacterTextSplitter(
      separator = "\n",   #우리가 가져온 텍스트가 무엇을 기준으로 나뉘어 있는지?
      chunk_size = 1000,  #올려둔 텍스트를 얼마만큼의 길이로 자를 것인가?
      chunk_overlap  = 200, #청크를 얼마나 겹쳐서 자를 것인가?
      length_function = len,
  )
  chunks = text_splitter.split_text(text)

  #임베딩
  embeddings = HuggingFaceBgeEmbeddings(model_name='BAAI/bge-m3')

  #FAISS VectorDB
  documents = FAISS.from_texts(chunks, embeddings)
  return documents

In [ ]:
pdf_path = '/content/프롬프트엔지니어링.pdf'

texts = ''
pdf_contents = PdfReader(pdf_path)

for page in pdf_contents.pages:
  texts += page.extract_text()

In [ ]:
texts

'프롬프트 엔지니어링 :\n업무능력의 혁신INDEX\n1프롬프트 엔지니어링이란 무엇인가 ?\n2프롬프트 엔지니어링의 구성요소\n3프롬프트 엔지니어링 스킬\n4프롬프트 엔지니어링 실습Generative AI?프롬프트 엔지니어링이란 ?\ndesigning affective inputs and fine-tuing parameters\nforaimodels togetthedesired results\n프롬프트 엔지니어링이란 ?\n①원하는결과물을 얻기위해\n②인풋,파라미터 ,아웃풋을\n③명확하게 ,정확하게 ,창의적으로\n④생성형 ai에게제시하는 것\n어학스킬+문제해결능력 +비판적사고프롬프트 엔지니어링의 구성요소\n1인풋(inputs)\n출력물의 기반이되는고유한데이터와 정보 .\n2파라미터 (parameter)\n작업지시 및맥락정보등\n사용자의 요청과명령을수행하는 데필요한행동 (작업지시 ),작업지시를\n이해하기 위한배경정보 (맥락 )등을의미\n3아웃풋 (outputs)\n결과물의 유형 ,형식 ,구조에대한제한과기대값 .\n원하는출력물의 형태를구체적으로 가이드해야 한다 .프롬프트 엔지니어링의 스킬\n1명령(task)\n2맥락(context)\n3페르소나 (persona)4예시(example)\n5포맷(format)\n6어조(tone)프롬프트 엔지니어링의 스킬\n1명령(task)1프롬프트 ,1명령\n서술어로 ,간결히\n~를제시해줘 .\n~를제안해줘 .프롬프트 엔지니어링의 스킬\n2맥락(context)현재어떤상황인지 ?\n의도와목표가무엇인지 ?\n우려되는 사항은무엇인지 ?\n답변에있어고려해야 할제약 /규칙은무엇인지 ?프롬프트 엔지니어링의 스킬\n3페르소나 (persona)롤플레이처럼 지시하기\n생성형 AI에게직무와역할을부여\n구체적인 전문가의 명칭을활용할것\n전문용어를포함할수록더전문적인 산출물을 냄프롬프트 엔지니어링의 스킬\n4예시(example)Few shot\n프롬프트 엔지니어링의 스킬\n5포맷(format)결과물의 형식및분량을구체적으로 지정\n결과물의 내용구성이나 아웃라인

In [ ]:
from langchain.embeddings import HuggingFaceBgeEmbeddings

docs = preprocess_text(texts)

/tmp/ipython-input-4023069445.py:14: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceBgeEmbeddings(model_name='BAAI/bge-m3')
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [ ]:
doc = docs.similarity_search("프롬프트 엔지니어링이 뭐야?")
print(doc)

[Document(id='ccc49de5-b2dd-4e59-af9c-45619b5462ee', metadata={}, page_content='프롬프트 엔지니어링 :\n업무능력의 혁신INDEX\n1프롬프트 엔지니어링이란 무엇인가 ?\n2프롬프트 엔지니어링의 구성요소\n3프롬프트 엔지니어링 스킬\n4프롬프트 엔지니어링 실습Generative AI?프롬프트 엔지니어링이란 ?\ndesigning affective inputs and fine-tuing parameters\nforaimodels togetthedesired results\n프롬프트 엔지니어링이란 ?\n①원하는결과물을 얻기위해\n②인풋,파라미터 ,아웃풋을\n③명확하게 ,정확하게 ,창의적으로\n④생성형 ai에게제시하는 것\n어학스킬+문제해결능력 +비판적사고프롬프트 엔지니어링의 구성요소\n1인풋(inputs)\n출력물의 기반이되는고유한데이터와 정보 .\n2파라미터 (parameter)\n작업지시 및맥락정보등\n사용자의 요청과명령을수행하는 데필요한행동 (작업지시 ),작업지시를\n이해하기 위한배경정보 (맥락 )등을의미\n3아웃풋 (outputs)\n결과물의 유형 ,형식 ,구조에대한제한과기대값 .\n원하는출력물의 형태를구체적으로 가이드해야 한다 .프롬프트 엔지니어링의 스킬\n1명령(task)\n2맥락(context)\n3페르소나 (persona)4예시(example)\n5포맷(format)\n6어조(tone)프롬프트 엔지니어링의 스킬\n1명령(task)1프롬프트 ,1명령\n서술어로 ,간결히\n~를제시해줘 .\n~를제안해줘 .프롬프트 엔지니어링의 스킬\n2맥락(context)현재어떤상황인지 ?\n의도와목표가무엇인지 ?\n우려되는 사항은무엇인지 ?\n답변에있어고려해야 할제약 /규칙은무엇인지 ?프롬프트 엔지니어링의 스킬\n3페르소나 (persona)롤플레이처럼 지시하기\n생성형 AI에게직무와역할을부여\n구체적인 전문가의 명칭을활용할것\n전문용어를포함할수록더전문적인 산출물을 냄프롬프트 엔지니어링의 스킬\n4예시(exa

In [ ]:
import os
from google.colab import userdata

os.environ['OPENAI_API_KEY'] = userdata.get('ssu')

In [ ]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0) #temperature(0~1)  1에 가까워질수록 '창의적'
chain = load_qa_chain(llm, chain_type="stuff") #stuff - 주어진 문서를 한번에 고려하여 작업하겠다.

#벡터DB, 리트리버를 전달 -> input_documents
response = chain.run(input_documents=doc, question="프롬프트 엔지니어링이 뭐야?")
print(response)

/tmp/ipython-input-1565859647.py:4: LangChainDeprecationWarning: This class is deprecated. See the following migration guides for replacements based on `chain_type`:
stuff: https://python.langchain.com/docs/versions/migrating_chains/stuff_docs_chain
map_reduce: https://python.langchain.com/docs/versions/migrating_chains/map_reduce_chain
refine: https://python.langchain.com/docs/versions/migrating_chains/refine_chain
map_rerank: https://python.langchain.com/docs/versions/migrating_chains/map_rerank_docs_chain

See also guides on retrieval and question-answering here: https://python.langchain.com/docs/how_to/#qa-with-rag
  chain = load_qa_chain(llm, chain_type="stuff") #stuff - 주어진 문서를 한번에 고려하여 작업하겠다.
/tmp/ipython-input-1565859647.py:7: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = chain.run(input_documents=doc, question="프롬프트 엔지니어링이 뭐야?")


프롬프트 엔지니어링은 원하는 결과물을 얻기 위해 인풋, 파라미터, 아웃풋을 명확하고 정확하게 창의적으로 생성형 AI에게 제시하는 것을 말합니다. 이는 업무능력의 혁신을 위한 기술로, 다양한 분야에서 활용 가능한 스킬셋입니다.


In [ ]:
#벡터 DB는 한번 만들고 계속 재활용
docs.save_local('/content/faiss')

In [ ]:
# 저장할 때 썼던 동일 모델로 임베딩 초기화
embeddings = HuggingFaceBgeEmbeddings(model_name='BAAI/bge-m3')

# 로컬에 저장된 DB 불러오기
vectorstore = FAISS.load_local(
    "/content/faiss",
    embeddings,
    allow_dangerous_deserialization=True  # colab/jupyter에서는 보통 이 옵션 필요
)

# 검색 테스트
result = vectorstore.similarity_search("프롬프트 엔지니어링에서 쓸 수 있는 스킬들 뭐 없어?", k=2)
print(result)

[Document(id='d986ea62-3bd5-4002-9079-4720f0e18d45', metadata={}, page_content='3페르소나 (persona)롤플레이처럼 지시하기\n생성형 AI에게직무와역할을부여\n구체적인 전문가의 명칭을활용할것\n전문용어를포함할수록더전문적인 산출물을 냄프롬프트 엔지니어링의 스킬\n4예시(example)Few shot\n프롬프트 엔지니어링의 스킬\n5포맷(format)결과물의 형식및분량을구체적으로 지정\n결과물의 내용구성이나 아웃라인을 제공\nhttps://sabarada.tistory.com /209프롬프트 엔지니어링의 스킬\n6어조(tone)가상의청중을제시(‘5살인것처럼설명해줘’)\n‘상냥하게 ’,‘친절하게 ’\n필요하다면 예시를제공후,예시의잘된부분을설명하여 가이드라인을 명확히프롬프트 엔지니어링의 스킬\n그외의팁\n1.영어를활용할것\n2.강조표시를활용할것\n3.결과물에 대해피드백을 제공하며 ,답변을순차적으로 이끌어낼것\n4.생성형 AI를협박할것.(보상을제시할것)프롬프트 엔지니어링 예제\n주제선정\n프롬프트 엔지니어링을 적용할업무주제를정한다 .\n프롬프트 엔지니어링 실습예제\n핵심요구사항 도출\n주제에대한핵심요구사항을 도출한다 .\n최적프롬프트 작성\n요구사항을 반영하여 프롬프트를 작성한다 .주제선정\n프롬프트 엔지니어링을 적용할업무주제를정한다 .\n핵심요구사항 도출\n주제에대한핵심요구사항을 도출한다 .\n“그려줘”\n그림에대한상세묘사\n예시제안 (fewshot)\n영어로\n?프롬프트 엔지니어링은\n업무생산성과 창의성을 높일수\n있는혁신적인 기술\n다양한분야에서 활용가능한\n실용적인 스킬셋\n윤리성과 품질관리에대한주\n의가필요\n결론및질의응답\n혁신적 실용적 주의!References\nPrompt Engineering For Everyone with ChatGPT and GPT -4,Andrei Gheorghiu\nPrompt Engineering for Generative AI ,James Phoenix, Mik